In [5]:
import pandas as pd
import numpy as np
import ast
import pickle

In [6]:
link1 = "./BD/df_tID.csv"
link2 = "./BD/df_nID.csv"
link3 = "./BD/df_tmdb_clean.csv"
link4 = "./BD/df_t_principals_clean.csv"
link5 = "./BD/df_basic_name_clean.csv"

link6 = "./BD/df_recherche.csv"

df_recherche = pd.read_csv(link6)

df_recherche['Titres possibles'] = df_recherche['Titres possibles'].apply(ast.literal_eval)
df_recherche['Genres'] = df_recherche['Genres'].apply(ast.literal_eval)
df_recherche['Liste acteurs'] = df_recherche['Liste acteurs'].apply(ast.literal_eval)
df_recherche['Réalisateurs'] = df_recherche['Réalisateurs'].apply(ast.literal_eval)


clé API OMDB: d0157019

Ajout d'une colonne "Liste Acteurs" et colonne "Réalisateurs" dans df_tmdb_clean

In [125]:
df_tID = pd.read_csv(link1)
df_nID = pd.read_csv(link2)
df_tmdb_clean = pd.read_csv(link3)
df_t_principals = pd.read_csv(link4)
df_basic_name_clean = pd.read_csv(link5)

In [31]:
#On filtre les categories du df_t_principals sur des nouveaux df pour un futur merge:
df_t_principals_act = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_t_principals_real = df_t_principals[df_t_principals['category'] == 'director']

#Préparation nom des personnes en fonction de leur ID:
df_basic_name_noms = df_basic_name_clean[['nconst', 'primaryName']]

#incorporation des noms aux différents df_t_principals:
df_liste_act = pd.merge(df_t_principals_act, df_basic_name_noms, how = 'left', on = 'nconst')
df_liste_real = pd.merge(df_t_principals_real, df_basic_name_noms, how = 'left', on = 'nconst')

#Il existe des NaN qui poseront problème par la suite, on peut les dropper car ils représentent à peine 1% du total:
df_liste_act = df_liste_act.dropna()
df_liste_real = df_liste_real.dropna()

#On drop les colonnes maintenant inutiles 'nconst' et 'category' et on groupby sur les tconst pour le merge:
df_liste_act = df_liste_act.drop(['nconst', 'category'], axis = 1).groupby('tconst').agg(list)
df_liste_real = df_liste_real.drop(['nconst', 'category'], axis = 1).groupby('tconst').agg(list)

#Premier merge pour la colonne "Liste Acteurs":
df_recherche = pd.merge(df_tmdb_clean, df_liste_act, how = 'left', left_on = 'ID', right_on = 'tconst').rename(columns = {'primaryName' : 'Liste acteurs'})

#2e pour l'ajout de la colonne "Réalisateurs":
df_recherche = pd.merge(df_recherche, df_liste_real, how = 'left', left_on = 'ID', right_on = 'tconst').rename(columns = {'primaryName' : 'Réalisateurs'})

#Il reste des lignes NaN, on les transforme en listes vides:
df_recherche = df_recherche.fillna("[]")

#Petit export csv pour pas refaire le code à chaque fois et garder transformer automatiquement les colonnes de listes en listes
df_recherche.to_csv("df_recherche.csv", index = False)

Maintenant On imagine que l'on demande à un utilisateur de rentrer des infos sur le film:

In [169]:
df_recherche.head(1)

,ID,Titre,Date de sortie,Durée (minutes),Genres,Langue Originale,Note moyenne,Nb de votes,Score de popularité,Résumé,Lien du poster,Budget,Titres possibles,Décénnie,Liste acteurs,Réalisateurs
0,tt0054823,Doctor Blood's Coffin,1961-01-01,92,[Horror],en,5.0,23,1.452,After being thrown out of medical school for e...,/7GkBT8cKPl1GhBuUnktJpjF1uV.jpg,0,"[Le cadavre qui tue, O Esquife do Morto-Vivo, ...",1960,"[Kieron Moore, Hazel Court, Ian Hunter, Kennet...",[Sidney J. Furie]


1. "Titre" en barre de recherche => choix = T
2. "Genre" en menu déroulant => choix = G
3. "Décénnie" en menu déroulant => choix = D
4. "Acteur" en barre recherche => choix = A
5. "Réalisateur" en barre recherche => choix = R

In [157]:
#Pour le test:
T = 'ボトムス ～最底で最強？な私たち～'
G = 'Horror'
D = 1990
A = 'Brad Pitt'
R = 'Christopher Nolan'

In [3]:
#A chaque lancement du code on créé un nouveau df pour cette recherche:
df_reponse = df_recherche.copy()
#Ensuite on mets des conditions en fonction du choix de l'utilisateur:
if T != np.nan:
    df_reponse = df_reponse[df_reponse['Titres possibles'].apply(lambda x: T in x)]
if G != np.nan:
    df_reponse = df_reponse[df_reponse['Genres'].apply(lambda x: G in x)]
if D != np.nan:
    df_reponse = df_reponse[df_reponse['Décénnie'] == D]
if A != np.nan:
    df_reponse = df_reponse[df_reponse['Liste acteurs'].apply(lambda x: A in x)]
if R != np.nan:
    df_reponse = df_reponse[df_reponse['Réalisateurs'].apply(lambda x: R in x)]   


NameError: name 'T' is not defined

On veut la liste complète des acteurs, des réalisateurs et des genres pour le système de recherche

In [11]:
#On créé une liste vide qui se remplira de tous les acteurs disponibles:
acteurs = []
reals = []
genres = []

#Fonction pour un futur "apply" qui ajoute chaque liste du df à la liste vide
def liste_acteurs(x):
    acteurs.extend(x)

def liste_reals(x):
    reals.extend(x)

def liste_genres(x):
    genres.extend(x)   

df_recherche['Liste acteurs'].apply(liste_acteurs)
df_recherche['Réalisateurs'].apply(liste_reals)
df_recherche['Genres'].apply(liste_genres)

#Pour supprimer les doublons on passe par un set:

acteurs = list(set(acteurs))
reals = list(set(reals))
genres = list(set(genres))
decennies = list(df_recherche['Décénnie'].unique())

In [12]:
with open('mes_listes.pkl', 'wb') as f:
    pickle.dump(acteurs, f)
    pickle.dump(reals, f)
    pickle.dump(genres, f)
    pickle.dump(decennies, f)

In [13]:
genres

['Science Fiction',
 'Drama',
 'Family',
 'TV Movie',
 'Action',
 'Fantasy',
 'Adventure',
 'Western',
 'Horror',
 'Documentary',
 'History',
 'Crime',
 'Animation',
 'Comedy',
 'Mystery',
 'Music',
 'Romance',
 'Thriller',
 'War']